## MySQL_Prepare
* Background Info + Schema/Data Preparation
---
### Introduction

* mysql deployed on AWS
* used faker for data

---
### Schema
| 表名         | 字段名               | 类型             | 说明                   |
|--------------|----------------------|------------------|------------------------|
| airlines     | airline_id            | INT (PK)         | 航空公司ID             |
|              | name                  | VARCHAR(255)     | 航空公司名称           |
|              | alias                 | VARCHAR(255)     | 别名                   |
|              | iata                  | VARCHAR(10)      | IATA代码               |
|              | icao                  | VARCHAR(10)      | ICAO代码               |
|              | country               | VARCHAR(100)     | 国家                   |
|              | active                | CHAR(1)          | 是否运营 (Y/N)         |
| airports     | airport_id            | INT (PK)         | 机场ID                 |
|              | name                  | VARCHAR(255)     | 机场名称               |
|              | city                  | VARCHAR(255)     | 城市                   |
|              | country               | VARCHAR(255)     | 国家                   |
|              | iata                  | VARCHAR(10)      | IATA代码               |
|              | icao                  | VARCHAR(10)      | ICAO代码               |
|              | latitude              | FLOAT            | 纬度                   |
|              | longitude             | FLOAT            | 经度                   |
|              | altitude              | INT              | 海拔                   |
|              | timezone              | FLOAT            | 时区                   |
|              | dst                   | CHAR(1)          | 夏令时规则             |
|              | tz_database_time_zone | VARCHAR(100)     | 时区数据库名称         |
| aircrafts    | aircraft_id           | INT (PK)         | 飞机ID                 |
|              | model                 | VARCHAR(255)     | 飞机型号               |
|              | manufacturer          | VARCHAR(255)     | 制造商                 |
|              | iata_code             | VARCHAR(10)      | IATA代码               |
|              | icao_code             | VARCHAR(10)      | ICAO代码               |
|              | seats                 | INT              | 座位数量               |
| routes       | route_id              | INT (PK, Auto Increment) | 航线ID      |
|              | airline_id            | INT (FK → airlines)      | 航空公司ID |
|              | source_airport_id     | INT (FK → airports)      | 起飞机场ID |
|              | destination_airport_id| INT (FK → airports)      | 降落机场ID |
|              | codeshare             | CHAR(1)                  | 是否共享航班 |
|              | stops                 | INT                      | 中途停留次数 |
|              | equipment             | VARCHAR(50)              | 飞机类型代码 |
| flights      | flight_id             | INT (PK, Auto Increment) | 航班ID         |
|              | route_id              | INT (FK → routes)        | 航线ID         |
|              | flight_number         | VARCHAR(20)              | 航班号         |
|              | departure_time        | TIME                     | 起飞时间       |
|              | arrival_time          | TIME                     | 到达时间       |
|              | flight_date           | DATE                     | 起飞日期       |
|              | aircraft_id           | INT (FK → aircrafts)      | 飞机ID         |
|              | status                | VARCHAR(50)              | 航班状态       |
| passengers   | passenger_id          | INT (PK)                 | 乘客ID         |
|              | first_name            | VARCHAR(100)             | 名             |
|              | last_name             | VARCHAR(100)             | 姓             |
|              | passport_number       | VARCHAR(50, UNIQUE)       | 护照号         |
|              | nationality           | VARCHAR(100)             | 国籍           |
|              | date_of_birth         | DATE                     | 出生日期       |
| bookings     | booking_id            | INT (PK, Auto Increment) | 订票ID         |
|              | passenger_id          | INT (FK → passengers)    | 乘客ID         |
|              | flight_id             | INT (FK → flights)       | 航班ID         |
|              | booking_date          | DATE                     | 订票日期       |
|              | seat_number           | VARCHAR(10)              | 座位号         |
|              | ticket_price          | DECIMAL(10,2)            | 票价           |

---
### Database Account
* aws
	* user:
	* password:
	* endpoint:


### Environment Setup

1. install Python Packages



In [ ]:
pip install pymysql sqlalchemy faker pandas

2. Connect to DB & Create Tables
    1. Will delete original DB before creating

In [45]:
# Connect to DB -----------------------------------------------------------------------
from sqlalchemy import create_engine, text
from faker import Faker
import pandas as pd

faker = Faker()

# connection info
username = ''
password = ''
host = ''
port = ''
database = ''

# 先连接到 mysql 系统库（因为要Drop/Create数据库）
engine_root = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/mysql",
    connect_args={"charset": "utf8mb4"}
)

# Drop + Create Database
with engine_root.connect() as conn:
    conn.execute(text(f"DROP DATABASE IF EXISTS {database};"))
    conn.execute(text(f"""
        CREATE DATABASE {database}
        DEFAULT CHARACTER SET utf8mb4
        DEFAULT COLLATE utf8mb4_unicode_ci;
    """))

print(f"✅ Database `{database}` dropped and recreated successfully!")

# connect to Flight DB
engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}",
    connect_args={"charset": "utf8mb4"}
)

# Create Tables -----------------------------------------------------------------------
tables = [
    # 1. Airlines
    """
    CREATE TABLE IF NOT EXISTS airlines (
        airline_id INT PRIMARY KEY, -- Unique ID for each airline
        name VARCHAR(255) NOT NULL, -- Airline name
        alias VARCHAR(255), -- Alternative name or abbreviation
        iata VARCHAR(10) UNIQUE, -- Unique IATA airline code
        icao VARCHAR(10) UNIQUE, -- Unique ICAO airline code
        country VARCHAR(100), -- Country where the airline is based
        active CHAR(1) DEFAULT 'Y' CHECK (active IN ('Y', 'N')) -- Whether the airline is active (Y/N)
    );
    """,

    # 2. Airports
    """
    CREATE TABLE IF NOT EXISTS airports (
        airport_id INT PRIMARY KEY, -- Unique ID for each airport
        name VARCHAR(255) NOT NULL, -- Airport name
        city VARCHAR(255) NOT NULL, -- City where the airport is located
        country VARCHAR(255) NOT NULL, -- Country where the airport is located
        iata VARCHAR(10) UNIQUE, -- Unique IATA airport code
        icao VARCHAR(10) UNIQUE, -- Unique ICAO airport code
        latitude FLOAT NOT NULL, -- Latitude coordinate
        longitude FLOAT NOT NULL, -- Longitude coordinate
        altitude INT, -- Altitude in feet
        timezone FLOAT, -- Timezone offset from UTC
        dst CHAR(1), -- Daylight saving time code
        tz_database_time_zone VARCHAR(100) -- Timezone database name
    );
    """,

    # 3. Aircrafts
    """
    CREATE TABLE IF NOT EXISTS aircrafts (
        aircraft_id INT PRIMARY KEY, -- Unique ID for each aircraft type
        model VARCHAR(255) NOT NULL, -- Aircraft model (e.g., A320, B737)
        manufacturer VARCHAR(255) NOT NULL, -- Manufacturer name (e.g., Airbus, Boeing)
        iata_code VARCHAR(10) UNIQUE, -- Unique IATA aircraft code
        icao_code VARCHAR(10) UNIQUE, -- Unique ICAO aircraft code
        seats INT NOT NULL -- Number of seats available
    );
    """,

    # 4. Routes
    """
    CREATE TABLE IF NOT EXISTS routes (
        route_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each route
        airline_id INT NOT NULL, -- Airline operating the route
        source_airport_id INT NOT NULL, -- Departure airport ID
        destination_airport_id INT NOT NULL, -- Arrival airport ID
        codeshare CHAR(1), -- Indicates if the route is a codeshare
        stops INT DEFAULT 0, -- Number of stops in the route
        equipment VARCHAR(50), -- Equipment or aircraft type used
        FOREIGN KEY (airline_id) REFERENCES airlines(airline_id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE,
        FOREIGN KEY (source_airport_id) REFERENCES airports(airport_id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE,
        FOREIGN KEY (destination_airport_id) REFERENCES airports(airport_id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE
    );
    """,

    # 5. Flights
    """
    CREATE TABLE IF NOT EXISTS flights (
        flight_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each flight
        route_id INT NOT NULL, -- Associated route ID
        flight_number VARCHAR(20) NOT NULL, -- Flight number (e.g., AA100)
        departure_time TIME NOT NULL, -- Scheduled departure time
        arrival_time TIME NOT NULL, -- Scheduled arrival time
        flight_date DATE NOT NULL, -- Date of the flight
        aircraft_id INT NOT NULL, -- Aircraft assigned to the flight
        status VARCHAR(50) DEFAULT 'Scheduled', -- Flight status (Scheduled, Delayed, etc.)
        FOREIGN KEY (route_id) REFERENCES routes(route_id)
            ON DELETE CASCADE
            ON UPDATE CASCADE,
        FOREIGN KEY (aircraft_id) REFERENCES aircrafts(aircraft_id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE
    );
    """,

    # 6. Passengers
    """
    CREATE TABLE IF NOT EXISTS passengers (
        passenger_id INT PRIMARY KEY, -- Unique ID for each passenger
        first_name VARCHAR(100) NOT NULL, -- Passenger's first name
        last_name VARCHAR(100) NOT NULL, -- Passenger's last name
        passport_number VARCHAR(50) UNIQUE NOT NULL, -- Unique passport number
        nationality VARCHAR(100) NOT NULL, -- Nationality of the passenger
        date_of_birth DATE NOT NULL -- Passenger's date of birth
    );
    """,

    # 7. Bookings
    """
    CREATE TABLE IF NOT EXISTS bookings (
        booking_id INT PRIMARY KEY AUTO_INCREMENT, -- Unique ID for each booking
        passenger_id INT NOT NULL, -- Passenger who made the booking
        flight_id INT NOT NULL, -- Flight booked
        booking_date DATE NOT NULL, -- Date when the booking was made
        seat_number VARCHAR(10) NOT NULL, -- Seat number assigned
        ticket_price DECIMAL(10,2) NOT NULL, -- Price paid for the ticket
        FOREIGN KEY (passenger_id) REFERENCES passengers(passenger_id)
            ON DELETE CASCADE
            ON UPDATE CASCADE,
        FOREIGN KEY (flight_id) REFERENCES flights(flight_id)
            ON DELETE CASCADE
            ON UPDATE CASCADE
    );
    """
]

# 循环建表
with engine.connect() as conn:
    for sql in tables:
        conn.execute(text(sql))

print("✅ All tables created successfully!")

✅ Database `Flight` dropped and recreated successfully!
✅ All tables created successfully!


3. Insert Fake Data

In [46]:
from faker import Faker
import pandas as pd

faker = Faker()

# 记录已经生成过的 iata_code 和 icao_code，避免重复
used_airline_iata = set()
used_airline_icao = set()
used_aircraft_iata = set()
used_aircraft_icao = set()
used_airport_iata = set()
used_airport_icao = set()

def generate_unique_code(used_set, pattern):
    code = faker.lexify(text=pattern).upper()
    while code in used_set:
        code = faker.lexify(text=pattern).upper()
    used_set.add(code)
    return code

# 1. Airlines
airlines = []
for i in range(1, 31):
    airlines.append({
        'airline_id': i,
        'name': faker.company(),
        'alias': faker.word(),
        'iata': generate_unique_code(used_airline_iata, '??'),
        'icao': generate_unique_code(used_airline_icao, '???'),
        'country': faker.country(),
        'active': faker.random_element(elements=('Y', 'N'))
    })

airlines_df = pd.DataFrame(airlines)
airlines_df.to_sql('airlines', con=engine, if_exists='append', index=False)

# 2. Airports
airports = []
for i in range(1, 31):
    airports.append({
        'airport_id': i,
        'name': faker.city() + " International Airport",
        'city': faker.city(),
        'country': faker.country(),
        'iata': generate_unique_code(used_airport_iata, '???'),
        'icao': generate_unique_code(used_airport_icao, '????'),
        'latitude': faker.latitude(),
        'longitude': faker.longitude(),
        'altitude': faker.random_int(min=0, max=5000),
        'timezone': faker.random_int(min=-12, max=12),
        'dst': faker.random_element(elements=('E', 'A', 'S', 'O', 'Z', 'N', 'U')),
        'tz_database_time_zone': faker.timezone()
    })

airports_df = pd.DataFrame(airports)
airports_df.to_sql('airports', con=engine, if_exists='append', index=False)

# 3. Aircrafts
aircrafts = []
for i in range(1, 31):
    aircrafts.append({
        'aircraft_id': i,
        'model': faker.word().capitalize() + " " + faker.random_element(elements=['737', '777', 'A320', 'A350']),
        'manufacturer': faker.company(),
        'iata_code': generate_unique_code(used_aircraft_iata, '??'),
        'icao_code': generate_unique_code(used_aircraft_icao, '???'),
        'seats': faker.random_int(min=50, max=400)
    })

aircrafts_df = pd.DataFrame(aircrafts)
aircrafts_df.to_sql('aircrafts', con=engine, if_exists='append', index=False)

# 4. Routes
airlines_ids = pd.read_sql("SELECT airline_id FROM airlines", con=engine)['airline_id'].tolist()
airports_ids = pd.read_sql("SELECT airport_id FROM airports", con=engine)['airport_id'].tolist()

routes = []
for i in range(1, 31):
    source, destination = faker.random_elements(elements=airports_ids, length=2, unique=True)
    routes.append({
        'route_id': i,
        'airline_id': faker.random_element(elements=airlines_ids),
        'source_airport_id': source,
        'destination_airport_id': destination,
        'codeshare': faker.random_element(elements=('Y', 'N')),
        'stops': faker.random_int(min=0, max=2),
        'equipment': faker.lexify(text='???')
    })

routes_df = pd.DataFrame(routes)
routes_df.to_sql('routes', con=engine, if_exists='append', index=False)

# 5. Flights
route_ids = pd.read_sql("SELECT route_id FROM routes", con=engine)['route_id'].tolist()
aircraft_ids = pd.read_sql("SELECT aircraft_id FROM aircrafts", con=engine)['aircraft_id'].tolist()

flights = []
for i in range(1, 61):
    flights.append({
        'flight_id': i,
        'route_id': faker.random_element(elements=route_ids),
        'flight_number': faker.bothify(text='??###').upper(),
        'departure_time': faker.time(),
        'arrival_time': faker.time(),
        'flight_date': faker.date_this_year(),
        'aircraft_id': faker.random_element(elements=aircraft_ids),
        'status': faker.random_element(elements=['Scheduled', 'Cancelled', 'Delayed', 'Departed'])
    })

flights_df = pd.DataFrame(flights)
flights_df.to_sql('flights', con=engine, if_exists='append', index=False)

# 6. Passengers
passengers = []
for i in range(1, 91):
    passengers.append({
        'passenger_id': i,
        'first_name': faker.first_name(),
        'last_name': faker.last_name(),
        'passport_number': faker.unique.bothify(text='??######'),
        'nationality': faker.country(),
        'date_of_birth': faker.date_of_birth(minimum_age=18, maximum_age=80)
    })

passengers_df = pd.DataFrame(passengers)
passengers_df.to_sql('passengers', con=engine, if_exists='append', index=False)

# 7. Bookings
passenger_ids = pd.read_sql("SELECT passenger_id FROM passengers", con=engine)['passenger_id'].tolist()
flight_ids = pd.read_sql("SELECT flight_id FROM flights", con=engine)['flight_id'].tolist()

bookings = []
for i in range(1, 121):
    bookings.append({
        'booking_id': i,
        'passenger_id': faker.random_element(elements=passenger_ids),
        'flight_id': faker.random_element(elements=flight_ids),
        'booking_date': faker.date_this_year(),
        'seat_number': faker.random_element(elements=['A', 'B', 'C', 'D', 'E', 'F']) + str(faker.random_int(min=1, max=30)),
        'ticket_price': faker.pydecimal(left_digits=3, right_digits=2, positive=True)
    })

bookings_df = pd.DataFrame(bookings)
bookings_df.to_sql('bookings', con=engine, if_exists='append', index=False)

120

4. Query Example

In [33]:
import pandas as pd

# 查询航线对应的航班数量
query = """
SELECT nationality, COUNT(*) AS count
FROM passengers
GROUP BY nationality
ORDER BY count DESC
LIMIT 10;
"""

# 用 pandas.read_sql
df = pd.read_sql(query, con=engine)
df.head(100)

,nationality,count
0,Algeria,3
1,Italy,3
2,Saudi Arabia,3
3,Rwanda,3
4,Togo,3
5,Ecuador,2
6,Zambia,2
7,Congo,2
8,Brunei Darussalam,2
9,Turks and Caicos Islands,2
